In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
import torch

In [3]:
raw_data = np.loadtxt("Iris_data\iris.data", delimiter=",", usecols=[0,1,2,3])[:100]
raw_label = np.genfromtxt("Iris_data\iris.data", delimiter=",", usecols=[4], dtype=str)[:100]
unique, raw_label = np.unique(raw_label, return_inverse=True)

## Torch implementation of logistics gradient and hessian

In [59]:
data = torch.tensor(np.c_[raw_data, np.ones(raw_data.shape[0])][:80], dtype=torch.double)
label = torch.tensor(label[:80])
w = torch.zeros(data.shape[1], dtype=torch.double, requires_grad=True)

In [60]:
l = torch.sum(-label*(data@w) + torch.log(1 + torch.exp(data@w)))

In [61]:
l

tensor(55.4518, dtype=torch.float64, grad_fn=<SumBackward0>)

In [62]:
grad = torch.autograd.grad(l, w, retain_graph=True, create_graph=True)

In [63]:
grad

(tensor([ 34.1000,  43.6000, -28.4000, -14.2000,  10.0000], dtype=torch.float64,
        grad_fn=<AddBackward0>),)

In [64]:
out = torch.tensor([], dtype=torch.double)
for i in grad[0]:
    out = torch.cat((out, torch.autograd.grad(i, w, retain_graph=True)[0]))
print(out)

tensor([593.2700, 342.8650, 290.4825,  77.4225, 108.1000, 342.8650, 206.9750,
        153.9850,  39.2050,  63.6500, 290.4825, 153.9850, 169.5250,  49.0475,
         50.8000,  77.4225,  39.2050,  49.0475,  14.9450,  13.2000, 108.1000,
         63.6500,  50.8000,  13.2000,  20.0000], dtype=torch.float64)


In [65]:
out = out.reshape(5,5)

In [66]:
torch.inverse(out)@grad[0]

tensor([ 0.0666,  0.5309, -0.9120, -0.8924,  1.3559], dtype=torch.float64,
       grad_fn=<MvBackward>)

## Sample implementation of logistics regression

In [4]:
import numpy as np
from sklearn import linear_model
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
np.random.seed(42)
import torch
from torch import nn,optim

In [7]:

class LogisticRegression:
    def __init__(self,max_iter=100,use_matrix=True):
        self.beta=None
        self.n_features=None
        self.max_iter=max_iter
        self.use_Hessian=use_matrix

    def fit(self,X,y):
        n_samples=X.shape[0]
        self.n_features=X.shape[1]
        extra=np.ones((n_samples,))
        X=np.c_[X,extra]
        self.beta=np.zeros((X.shape[1],))
        for i in range(self.max_iter):
            if self.use_Hessian is not True:                
                dldbeta=self._dldbeta(X,y,self.beta)                
                dldldbetadbeta=self._dldldbetadbeta(X,self.beta)
                self.beta-=(1./dldldbetadbeta*dldbeta)
            else:
                print(i," Iter:")
                dldbeta = self._dldbeta(X, y, self.beta)
                print("Gradient:", dldbeta)
                dldldbetadbeta = self._dldldbetadbeta_matrix(X, self.beta)
                # print("Hessian:\n", dldldbetadbeta)
                self.beta -= (np.linalg.inv(dldldbetadbeta).dot(dldbeta))
                print("#########################################################")



    @staticmethod
    def _dldbeta(X,y,beta):
        # 《机器学习》 公式 3.30
        m=X.shape[0]
        sum=np.zeros(X.shape[1],).T
        for i in range(m):
            temp=X[i]*(y[i]-np.exp(X[i].dot(beta))/(1+np.exp(X[i].dot(beta))))
            sum+=temp
        return -sum

    @staticmethod
    def _dldldbetadbeta_matrix(X,beta):
        m=X.shape[0]
        Hessian=np.zeros((X.shape[1],X.shape[1]))
        for i in range(m):
            p1 = np.exp(X[i].dot(beta)) / (1 + np.exp(X[i].dot(beta)))
            tmp=X[i].reshape((-1,1))
            Hessian+=tmp.dot(tmp.T)*p1*(1-p1)
        return Hessian

    @staticmethod
    def _dldldbetadbeta(X,beta):
        # 《机器学习》公式 3.31
        m=X.shape[0]
        sum=0.
        for i in range(m):
            p1=np.exp(X[i].dot(beta))/(1+np.exp(X[i].dot(beta)))
            sum+=X[i].dot(X[i].T)*p1*(1-p1)
        return sum

    def predict_proba(self,X):
        n_samples = X.shape[0]
        extra = np.ones((n_samples,))
        X = np.c_[X, extra]
        if self.beta is None:
            raise RuntimeError('cant predict before fit')
        p1 = np.exp(X.dot(self.beta)) / (1 + np.exp(X.dot(self.beta)))
        p0 = 1 - p1
        return np.c_[p0,p1]

    def predict(self,X):
        p=self.predict_proba(X)
        res=np.argmax(p,axis=1)
        return res

In [6]:
X_train = raw_data[:80]
y_train = raw_label[:80]
tinyml_logisticreg = LogisticRegression(max_iter=100,use_matrix=True)
tinyml_logisticreg.fit(X_train, y_train)

0  Iter:

Gradient:
 [ 34.1  43.6 -28.4 -14.2  10. ]
#########################################################
1  Iter:

Gradient:
 [ 7.29474076 10.16493666 -7.61727457 -3.67118444  2.25290718]
#########################################################
2  Iter:

Gradient:
 [ 2.39325624  3.51588204 -2.83873108 -1.34368637  0.76410355]
#########################################################
3  Iter:

Gradient:
 [ 0.82142273  1.26253837 -1.0812764  -0.50493471  0.27027994]
#########################################################
4  Iter:

Gradient:
 [ 0.28440728  0.45801751 -0.41467947 -0.19129009  0.09667873]
#########################################################
5  Iter:

Gradient:
 [ 0.09817673  0.16656197 -0.15938913 -0.07266519  0.03464679]
#########################################################
6  Iter:

Gradient:
 [ 0.0337334   0.06061194 -0.06120021 -0.02759457  0.01241696]
#########################################################
7  Iter:

Gradient:
 [ 0.01159072  0.022086

## Scikit-learn logistic Regression

In [4]:
from sklearn.linear_model import LogisticRegression
import numpy as np

In [5]:
data = np.genfromtxt("Abalone_data/abalone.data", delimiter=",", dtype=str)

# Remove third class
data = data[~(data[:, 0] == 'I')]
np.random.shuffle(data)

x = data[:,1:].astype(float)
label = data[:, 0]
unique, y = np.unique(label, return_inverse=True)

In [6]:
# Train test split
partition = 0.8
train_size = int(data.shape[0] * 0.8)

train_data = x[:train_size]
train_label = y[:train_size]

test_data = x[train_size:]
test_label = y[train_size:]

In [7]:
model = LogisticRegression()
model.fit(train_data, train_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [9]:
pred = model.predict(test_data)

In [11]:
print(((pred == test_label).astype(int).sum())/len(pred))

0.5537918871252204
